# Word2Vec model visualization


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from pandas import Series
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import os
import nltk
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ZuBaiR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Dataset Import

In [2]:
df ='health.harvard.edu.xlsx'
df = pd.read_excel(df)

In [3]:
df.head()

,article_date,article_title,article_content,article_url
0,2019-09-28,Popular heartburn drug ranitidine recalled: Wh...,If you or a family member take ranitidine (Zan...,https://www.health.harvard.edu/blog/popular-he...
1,2019-09-27,Is there a test for Alzheimer‚Äôs disease?,After spending 30 minutes hunting for your car...,https://www.health.harvard.edu/blog/is-there-a...
2,2019-09-26,Vitiligo: More than skin deep,Vitiligo (pronounced vit-uh-LIE-go) is a medic...,https://www.health.harvard.edu/blog/vitiligo-m...
3,2019-09-25,What donor offspring seek when they do DNA tes...,I wrote previously about parents who fear that...,https://www.health.harvard.edu/blog/what-donor...
4,2019-09-24,Weekend catch-up sleep won‚Äôt fix the effects...,Sleeping in late on a Saturday sounds deliciou...,https://www.health.harvard.edu/blog/weekend-ca...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1910 entries, 0 to 1909
Data columns (total 4 columns):
article_date       1910 non-null datetime64[ns]
article_title      1910 non-null object
article_content    1910 non-null object
article_url        1902 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 59.8+ KB


In [5]:
df.describe()

,article_date,article_title,article_content,article_url
count,1910,1910,1910,1902
unique,1662,1891,1907,1902
top,2009-03-30 00:00:00,Active surveillance is safe for low-risk prost...,"The answer is more friend than foe, if the fis...",https://www.health.harvard.edu/blog/can-exerci...
freq,23,2,2,1
first,2009-02-03 00:00:00,NaN,NaN,NaN
last,2019-09-28 00:00:00,NaN,NaN,NaN


In [6]:
df.describe(include=['O'])

,article_title,article_content,article_url
count,1910,1910,1902
unique,1891,1907,1902
top,Active surveillance is safe for low-risk prost...,"The answer is more friend than foe, if the fis...",https://www.health.harvard.edu/blog/can-exerci...
freq,2,2,1


In [7]:
article_content=[]


In [14]:
article_content=df['article_content']

In [15]:
article_content

0       If you or a family member take ranitidine (Zan...
1       After spending 30 minutes hunting for your car...
2       Vitiligo (pronounced vit-uh-LIE-go) is a medic...
3       I wrote previously about parents who fear that...
4       Sleeping in late on a Saturday sounds deliciou...
                              ...                        
1905    Editor in Chief Marc B. Garnick, M.D., discuss...
1906    Consider this scenario: A man diagnosed with p...
1907    At least initially, the pathology report is on...
1908    Until recently, most professionals have been s...
1909    In the spring of 1996, Andr√© James* was under...
Name: article_content, Length: 1910, dtype: object

In [16]:
for text in article_content:
    text = re.sub(r"^https://t.co/[a-zA-Z0-9]*\s", " ", text)
    text = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", " ", text)
    text = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", " ", text)
    text = text.lower()
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain't","am not",text)
    text = re.sub(r"wouldn't","would not",text)
    text = re.sub(r"shouldn't","should not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"couldn't","could not",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"\W"," ",text)
    text = re.sub(r"\d"," ",text)
    text = re.sub(r"\s+[a-z]\s+"," ",text)
    text = re.sub(r"\s+[a-z]$"," ",text)
    text = re.sub(r"^[a-z]\s+"," ",text)
    text = re.sub(r"\s+"," ",text)

In [17]:
# Preparing the dataset
sentences = nltk.sent_tokenize(text)

sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]
    

In [18]:
# Training the Word2Vec model
model = Word2Vec(sentences, min_count=1)

words = model.wv.vocab

words

{'spring': <gensim.models.keyedvectors.Vocab at 0x18e6f181508>,
 'andr': <gensim.models.keyedvectors.Vocab at 0x18e6f181548>,
 'james': <gensim.models.keyedvectors.Vocab at 0x18e6f181588>,
 'great': <gensim.models.keyedvectors.Vocab at 0x18e6f1815c8>,
 'deal': <gensim.models.keyedvectors.Vocab at 0x18e6f181708>,
 'stress': <gensim.models.keyedvectors.Vocab at 0x18e6f1818c8>,
 'married': <gensim.models.keyedvectors.Vocab at 0x18e6f181948>,
 'young': <gensim.models.keyedvectors.Vocab at 0x18e6f1819c8>,
 'family': <gensim.models.keyedvectors.Vocab at 0x18e6f181688>,
 'support': <gensim.models.keyedvectors.Vocab at 0x18e6f1817c8>,
 'finishing': <gensim.models.keyedvectors.Vocab at 0x18e6f181a88>,
 'medical': <gensim.models.keyedvectors.Vocab at 0x18e6f181ac8>,
 'training': <gensim.models.keyedvectors.Vocab at 0x18e6f181b88>,
 'big': <gensim.models.keyedvectors.Vocab at 0x18e6f181bc8>,
 'city': <gensim.models.keyedvectors.Vocab at 0x18e6f181c48>,
 'hospital': <gensim.models.keyedvectors.Voc

In [21]:
# Finding Word Vectors
vector = model.wv['medical']



In [22]:
vector

array([-0.00215386,  0.00354803, -0.00283695, -0.0020865 , -0.00331066,
       -0.0023061 , -0.0038936 ,  0.00326824, -0.00073591, -0.00045124,
       -0.00420544, -0.00212071,  0.00075786, -0.00027578, -0.00162796,
       -0.00188728,  0.00306596, -0.00414832, -0.00367658,  0.00402533,
       -0.00172711,  0.00010558,  0.00297452, -0.00214289, -0.00028687,
        0.00392303, -0.00105715, -0.00194613, -0.00149273,  0.00135238,
        0.00429532, -0.0005026 ,  0.00155667, -0.0007839 ,  0.00490922,
        0.00150254,  0.00032576, -0.00466301, -0.00280413,  0.00466286,
       -0.00208755,  0.00349672,  0.00410814,  0.00331383,  0.00436368,
       -0.00422367,  0.00138409,  0.00069102, -0.00157484,  0.00258874,
        0.0040431 , -0.00416052, -0.00440752,  0.00237482, -0.00114536,
        0.00106371,  0.00389188,  0.00471134, -0.00148999,  0.00136108,
       -0.0029275 , -0.00050937, -0.0016533 , -0.00489136,  0.00396967,
       -0.00250315, -0.00201858, -0.0029877 , -0.00511542,  0.00

In [24]:
# Most similar words
similar = model.wv.most_similar('medical')

In [25]:
similar

[('brigham', 0.31378671526908875),
 ('retention', 0.29472261667251587),
 ('flare', 0.2529670298099518),
 ('device', 0.25129979848861694),
 ('along', 0.24998563528060913),
 ('carry', 0.2467144876718521),
 ('help', 0.23849782347679138),
 ('year', 0.2380027025938034),
 ('syndrome', 0.23421652615070343),
 ('stimulator', 0.2313029021024704)]